## Realization in Tensorflow (2 conv + 2 fully-connected)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
%%time


x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])  #[-1, 28, 28, 3] for RGB

# convolutional layer; size of kernel = 5x5; 1 channel; number of filters - 32; 
W_conv_1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))

conv_1 = tf.nn.conv2d(x_image, W_conv_1, strides=[1, 1, 1, 1], padding="SAME") + b_conv_1
h_conv_1 = tf.nn.relu(conv_1)

# subsampling layer; ksize - size of window from which max element will be chosen
h_pool_1 = tf.nn.max_pool(h_conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Convolutional layer with 64 filters
W_conv_2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv_2 = tf.nn.conv2d(h_pool_1, W_conv_2, strides=[1, 1, 1, 1], padding="SAME") + b_conv_2

h_conv_2 = tf.nn.relu(conv_2)
h_pool_2 = tf.nn.max_pool(h_conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# convert 2d layer in flat layer
h_pool_2_flat = tf.reshape(h_pool_2, [-1, 7*7*64]) #28/2 = 14/2 = 7, 64 filters at last pool layer

# add fully-connected layer with 1024 neurons
W_fc_1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc_1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc_1 = tf.nn.relu(tf.matmul(h_pool_2_flat, W_fc_1) + b_fc_1)

# regulaize with drop-out layer
keep_probability = tf.placeholder(tf.float32)
h_fc_1_drop = tf.nn.dropout(h_fc_1, keep_probability)

# add second full-connected layer with 10 outs
W_fc_2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc_2 = tf.Variable(tf.constant(0.1, shape=[10]))

logit_conv = tf.matmul(h_fc_1_drop, W_fc_2) + b_fc_2
y_conv = tf.nn.softmax(logit_conv)

# define a error and add optimizer
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit_conv, labels=y))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

# estimate accuracy
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# start train
for i in range(10000):
    if i % 500 == 0 :
        print("Number of iteration : {}".format(i))
    batch_xs, batch_ys = mnist.train.next_batch(64)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_probability: 0.5})

Number of iteration : 0
Number of iteration : 500
Number of iteration : 1000
Number of iteration : 1500
Number of iteration : 2000
Number of iteration : 2500
Number of iteration : 3000
Number of iteration : 3500
Number of iteration : 4000
Number of iteration : 4500
Number of iteration : 5000
Number of iteration : 5500
Number of iteration : 6000
Number of iteration : 6500
Number of iteration : 7000
Number of iteration : 7500
Number of iteration : 8000
Number of iteration : 8500
Number of iteration : 9000
Number of iteration : 9500
Wall time: 6min 53s


In [13]:
acc_1 = sess.run(accuracy, feed_dict={x: mnist.test.images[:5000, :], y: mnist.test.labels[:5000, :], keep_probability: 1.})
acc_2 = sess.run(accuracy, feed_dict={x: mnist.test.images[5000:, :], y: mnist.test.labels[5000:, :], keep_probability: 1.})

print("Accuracy = {:.4f}".format((acc_1 + acc_2)/2.))

Accuracy = 0.9913


## Realization of the same CNN in Keras

In [34]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [35]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [36]:
%%time

# reshape image in 28*28*1 size
batch_size, img_rows, img_cols = 64, 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# convert in float and normalize to scale (0, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

# perform ohe-hot encoding
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# create model
model = Sequential()

# add first convolutional layer (32 filter)
model.add(Convolution2D(32, (5, 5), padding="same", input_shape=input_shape))
model.add(Activation('relu'))

# add fist pool layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

# add second convolutional layer (32 filter)
model.add(Convolution2D(64, (5, 5), padding="same", input_shape=input_shape))
model.add(Activation('relu'))

# add seond pool layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

# convert to flat layer
model.add(Flatten())

# add fully-connected layer with 1024 neurons and relu activation 
model.add(Dense(1024))
model.add(Activation('relu'))

# add dropout layer
model.add(Dropout(0.5))

# output layer with softmax function
model.add(Dense(10))
model.add(Activation('softmax'))

# define loss, optimizer and metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# start fitting
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10, verbose=1, validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)

print("Test score : {:.4f}".format(score[0]))
print("Test accuracy : {:.4f}".format(score[1]))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 71s 1ms/step - loss: 0.1266 - acc: 0.9600 - val_loss: 0.0312 - val_acc: 0.9891
Epoch 2/10
60000/60000 [==============================] - 70s 1ms/step - loss: 0.0426 - acc: 0.9869 - val_loss: 0.0277 - val_acc: 0.9911
Epoch 3/10
60000/60000 [==============================] - 69s 1ms/step - loss: 0.0297 - acc: 0.9908 - val_loss: 0.0242 - val_acc: 0.9925
Epoch 4/10
60000/60000 [==============================] - 70s 1ms/step - loss: 0.0234 - acc: 0.9927 - val_loss: 0.0241 - val_acc: 0.9914
Epoch 5/10
60000/60000 [==============================] - 70s 1ms/step - loss: 0.0193 - acc: 0.9935 - val_loss: 0.0238 - val_acc: 0.9926
Epoch 6/10
60000/60000 [==============================] - 70s 1ms/step - loss: 0.0143 - acc: 0.9955 - val_loss: 0.0254 - val_acc: 0.9921
Epoch 7/10
60000/60000 [==============================] - 70s 1ms/step - loss: 0.0140 - acc: 0.9954 - val_loss: 0.0242 - val_acc